In [2]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install -q -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 66.6 MB/s eta 0:00:00


In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import json
import re

# Initialize the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xxl")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-xxl")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

...

model-00003-of-00005.safetensors:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/6.06G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [2]:
import pandas as pd
import json

dataset = pd.read_csv("data_full_v1.csv")
dataset['ground_truth'] = dataset.apply(lambda row: 1 if row['label']=='yes' else 0, axis=1)
dataset = dataset.sample(frac=1)
dataset = json.loads(dataset.to_json(orient ='records'))

In [17]:
# Format the COPA query for the GPT-3.5 Turbo model
def format_query(item):
    # return f"{item['prompt']}. Yes or No?"
    # return f"{item['prompt']}. No or Yes?"
    return f"{item['prompt']}"

  # return f"{item['prompt']}. Output only single digit 0 or 1. 0 for no and 1 for yes. Output should have only single digit. Do not output any text other than a single digit"

format_query(dataset[0])

'Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationships: Zuph has a direct effect on rixq and xevu. Rixq has a direct effect on xevu.\nWe know that zuph causes rixq. zuph and rixq causes xevu. We observed an individual is zuph.\nWould an individual is xevu if rixq instead of not rixq?'

In [22]:
inputs = tokenizer.encode(format_query(dataset[1]), return_tensors="pt")
outputs = model.generate(inputs, max_length=40, num_beams=4, early_stopping=True)
# outputs = model.generate(inputs, max_length=40, num_beams=4)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

no


In [25]:
from tenacity import retry, stop_after_attempt, wait_random_exponential, RetryError
import re

correct_count = 0
processed_items = 0
processed = []
failed_items = []
y_true = []
y_pred = []

# Evaluate the accuracy of GPT-3.5 Turbo on the COPA dataset
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(1))
def evaluate_accuracy(item):
      global correct_count
      query = format_query(item)

      inputs = tokenizer.encode(query, return_tensors="pt")
      outputs = model.generate(inputs, max_length=40, num_beams=4, early_stopping=True)
      generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
      print(generated_text)




#       print(generated_text)
      generated_text = generated_text.removeprefix(query).lower()

      # print(generated_text)
      generated_text = [w for w in re.split('\W', generated_text) if w]
#       print(generated_text)
      # generated_text = ''.join(filter(str.isalnum, generated_text)).split()

      if (('yes' in generated_text) or ('yes.' in generated_text)) and ((not 'no' in generated_text) and (not 'no.' in generated_text)):
        response = 1
      elif (('no' in generated_text) or ('no.' in generated_text)) and ((not 'yes' in generated_text) and (not 'yes.' in generated_text)):
        response = 0
      else:
#         print(query)
        # print(response)
#         print(generated_text)
        raise Exception("failure")

      # response = pred[-1]
      print(response)
      # print(item['ground_truth'])
      correct_hypothesis = ((int(response) == item['ground_truth'])==True)
      y_true.append(item['ground_truth'])
      y_pred.append(int(response))
      # print(correct_hypothesis)
      correct_count += correct_hypothesis
      print("correct_count is ", correct_count)
      # print("completed\n")


# Evaluate accuracy
def evaluate_accuracy_wrapper(data):
      global correct_count
      global processed_items
      global failed_items
      for item in data:
          try:
              evaluate_accuracy(item)
              processed_items = processed_items +1
              processed.append(item)
          except RetryError as e:
              print("failed")
              failed_items.append(item)
          if processed_items==1000:
            break
          if processed_items%10==0:
            print('processed_items ', processed_items)
      print("correct_count in wrapper is ", correct_count)
      return correct_count / len(data)

accuracy = evaluate_accuracy_wrapper(dataset)
print(f"Accuracy: {accuracy * 100:.2f}%")

yes
1
correct_count is  1
no
0
correct_count is  2
yes
1
correct_count is  3
no
0
correct_count is  3
it is not possible to tell
failed
no
0
correct_count is  3
yes
1
correct_count is  4
yes
1
correct_count is  4
yes
1
correct_count is  5
yes
1
correct_count is  6
no
0
correct_count is  7
processed_items  10
yes
1
correct_count is  8
yes
1
correct_count is  8
yes
1
correct_count is  8
yes
1
correct_count is  8
no
0
correct_count is  8
yes
1
correct_count is  9
yes
1
correct_count is  10
yes
1
correct_count is  11
yes
1
correct_count is  11
no
0
correct_count is  11
processed_items  20
yes
1
correct_count is  11
no
0
correct_count is  11
yes
1
correct_count is  12
no
0
correct_count is  13
no
0
correct_count is  14
decreases
failed
no
0
correct_count is  15
no
0
correct_count is  16
yes
1
correct_count is  16
yes
1
correct_count is  16
no
0
correct_count is  16
processed_items  30
yes
1
correct_count is  16
yes
1
correct_count is  17
no
0
correct_count is  18
yes
1
correct_count is  18


In [26]:
with open("y_true.txt", "w") as f:
    for s in y_true:
        f.write(str(s) +"\n")

In [27]:
with open("y_pred.txt", "w") as f:
    for s in y_pred:
        f.write(str(s) +"\n")

In [28]:
with open("processed.txt", "w") as f:
    for s in processed:
        f.write(str(s) +"\n")

In [29]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.61      0.62      0.62       513
           1       0.59      0.59      0.59       487

    accuracy                           0.60      1000
   macro avg       0.60      0.60      0.60      1000
weighted avg       0.60      0.60      0.60      1000

